In [ ]:
import sys

sys.path.append(r"e:\Data_Science")

# Without Header

In [ ]:
import csv

with open('../test.txt') as f:
    tab_reader = csv.reader(f)
    for row in tab_reader:
        date = row[0]
        symbol = row[1]
        closing_price = float(row[2])
        print(date, symbol, closing_price)

# With Headers

In [ ]:
import csv

with open('../test.txt') as f:
    tab_reader = csv.DictReader(f, fieldnames=['date', 'symbol', 'closing_price'])
    for row in tab_reader:
        date = row['date']
        symbol = row['symbol']
        closing_price = float(row['closing_price'])

In [ ]:
from bs4 import BeautifulSoup
import requests

url = ("https://raw.githubusercontent.com/joelgrus/data/master/getting-data.html")

html = requests.get(url).text
soup = BeautifulSoup(html, 'html5lib')

first_paragraph = soup.find('p')
first_paragraph_text = soup.p.text
first_paragraph_words = soup.p.text.split()

first_paragraph_id = soup.p.get('id')
first_paragraph_id2 = soup.p['id']

all_paragraphs = soup.find_all('p')
paragraphs_with_ids = [p for p in soup('p') if p.get('id')]

print(all_paragraphs)






# Keeping Tabs on Congress

In [ ]:
from bs4 import BeautifulSoup
import requests

url = ("https://www.house.gov/representatives")
text = requests.get(url).text
soup = BeautifulSoup(text, 'html5lib')

all_urls = [a['href'] 
            for a in soup('a')
            if a.has_attr('href')]

print(len(all_urls))

In [ ]:
import re

# Must start with http:// or https://
# Must end with .house.gov or .house.gov/
regex = r"https?://.*\.house\.gov/?$"


good_urls = [url for url in all_urls if re.match(regex, url)]

good_urls = list(set(good_urls))
print(len(good_urls))

html = requests.get('https://jayapal.house.gov').text
soup = BeautifulSoup(html, 'html5lib')

links ={a['href'] for a in soup('a') if 'press releases' in a.text.lower()}

print(links)

In [ ]:
from typing import Dict, Set

press_releases: Dict[str, Set[str]] = {}

for house_url in good_urls:
    html = requests.get(house_url).text
    soup = BeautifulSoup(html, 'html5lib')
    pr_links = {a['href'] for a in soup('a') if 'press releases' in a.text.lower()}

    press_releases[house_url] = pr_links

In [ ]:
def paragraph_mentions(text: str, keyword: str) -> bool:
    """
    Returns True if a <p> inside the text mentions {keyword}
    """
    soup = BeautifulSoup(text, 'html5lib')
    paragraphs = [p.get_text() for p in soup('p')]

    return any(keyword.lower() in paragraph.lower() for paragraph in paragraphs)

text = """<body><h1>Facebook</h1><p>Twitter</p>"""
assert paragraph_mentions(text, "twitter")
assert not paragraph_mentions(text, "facebook")


In [ ]:
for house_url, pr_links in press_releases.items():
    for pr_link in pr_links:
        url = f"{house_url}/{pr_link}"
        text = requests.get(url).text

        if paragraph_mentions(text, 'data'):
            print(f"{house_url}")
            break

In [ ]:
import json
serialised = """{"title": "Data Science Book",
                    "author": "Joel Grus",
                    "publicationYear": 2019,
                    "topics": ["data", "science", "data science"]}"""

# parse the JSON to create a Python dict
deserialised = json.loads(serialised)
assert deserialised["publicationYear"] == 2019
assert "data science" in deserialised["topics"]



In [ ]:
import requests, json

github_user = "RepusNamuh"
endpoint = f"https://api.github.com/users/{github_user}/repos"

repos = json.loads(requests.get(endpoint).text)

In [ ]:
from collections import Counter
from dateutil.parser import parse

dates = [parse(repo["created_at"]) for repo in repos]
month_counts = Counter(date.month for date in dates)
weekday_counts = Counter(date.weekday() for date in dates)

last_5_repositories = sorted(repos, key=lambda r: r["pushed_at"], reverse=True)[:5]

last_5_languages = [repo["language"] for repo in last_5_repositories]

repos_name = [repo['name'] for repo in repos]
print(repos_name)

print(last_5_languages)

In [1]:

import os


CONSUMER_KEY = os.getenv("Twitter_CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("Twitter_CONSUMER_SECRET")

print(CONSUMER_KEY, CONSUMER_SECRET)

NS21BKG4cozndD0kRIveN3lIq 5Nc46yo3LxD7OdwJMRQ07YFEH0vRCdrdtt6zNw5kl1rsp7meZe


In [2]:
from math import pi
import webbrowser
from twython import Twython

temp_client = Twython(CONSUMER_KEY, CONSUMER_SECRET)
temp_creds = temp_client.get_authentication_tokens()
url = temp_creds['auth_url']

print(f"go visit {url} and get the pin code and paste it here")
webbrowser.open(url)
PIN_CODE = input("Enter your pin code: ")

auth_client = Twython(CONSUMER_KEY,
                        CONSUMER_SECRET,
                        temp_creds['oauth_token'],
                        temp_creds['oauth_token_secret'])

final_step = auth_client.get_authorized_tokens(PIN_CODE)

ACCESS_TOKEN = final_step['oauth_token']
ACCESS_TOKEN_SECRET = final_step['oauth_token_secret']

# And get a new Twython instance using them.
twitter = Twython(CONSUMER_KEY,
                    CONSUMER_SECRET,
                    ACCESS_TOKEN,
                    ACCESS_TOKEN_SECRET)



go visit https://api.twitter.com/oauth/authenticate?oauth_token=6r0fbAAAAAABxEpPAAABk0d9aAw and get the pin code and paste it here


In [21]:
from twython import TwythonStreamer

# Appending data to a global variable is pretty poor form
# but it makes the example much simpler
tweets = []

class MyStreamer(TwythonStreamer):
    def on_success(self, data):
        """
        What do we do when Twitter sends us data?
        Here data will be a Python dict representing a tweet
        """
        # We only want to collect English-language tweets
        if data.get('lang') == 'en':
            tweets.append(data)
            print(f"received tweet #{len(tweets)}")

        # Stop when we've collected enough
        if len(tweets) >= 100:
            self.disconnect()

    def on_error(self, status_code, *data):
        print(status_code, data)
        self.disconnect()

In [27]:

stream = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET,
                    ACCESS_TOKEN, ACCESS_TOKEN_SECRET)



stream.statuses.filter(track='star rail')

404 (b'', {'perf': '7402827104', 'cache-control': 'no-cache, no-store, max-age=0', 'content-length': '0', 'x-transaction-id': '353e823a6638589c', 'x-response-time': '93', 'x-connection-hash': '6f8c6598ef6be86b05ed2d47275d05b45a7f1b7e9b655f5cb214ff77275521ed', 'date': 'Wed, 20 Nov 2024 03:05:24 GMT', 'server': 'tsa_m'})


In [26]:
from collections import Counter

print(tweets)
top_hashtags = Counter(hashtag['text'].lower()
                        for tweet in tweets
                        for hashtag in tweet["entities"]["hashtags"])

print(top_hashtags.most_common(5))

[]
[]
